## Hashing

Hashing data is a common operation that is facilitated
through Deno's support for the Web Crypto API. In addition,
the Deno standard library's implementation extends the standard API, allowing for
more advanced uses.


In our first example, we'll hash the contents of a string variable.


In [1]:
const message = "The easiest, most secure JavaScript runtime.";
message;


"The easiest, most secure JavaScript runtime."

Before we can pass our message to the hashing function, we first need to encode it into a uint8 array.


In [2]:
const messageBuffer = new TextEncoder().encode(message);
messageBuffer;


Uint8Array(44) [
   84, 104, 101,  32, 101,  97, 115, 105, 101,
  115, 116,  44,  32, 109, 111, 115, 116,  32,
  115, 101,  99, 117, 114, 101,  32,  74,  97,
  118,  97,  83,  99, 114, 105, 112, 116,  32,
  114, 117, 110, 116, 105, 109, 101,  46
]

Here, we use the built-in `crypto.subtle.digest` method to hash our original message.
The hash is returned as an `ArrayBuffer`. To obtain a string
we'll need to do a little more work.


In [3]:
const hashBuffer = await crypto.subtle.digest("SHA-256", messageBuffer);
hashBuffer;


ArrayBuffer {
  [Uint8Contents]: <1a c7 d6 a8 8e 74 e6 3d 1e 2c 28 70 fa 6e 80 8c 4e d6 b4 27 27 70 bf c2 2c a1 bb a7 e1 16 79 20>,
  byteLength: 32
}

We can decode this into a string using the standard
library's `toHashString` method.


In [4]:
import { toHashString } from "https://deno.land/std@0.194.0/crypto/to_hash_string.ts";
const hash = toHashString(hashBuffer);
console.log(hash);


1ac7d6a88e74e63d1e2c2870fa6e808c4ed6b4272770bfc22ca1bba7e1167920


For our second example, we'll hash the contents of a file.
Hashing a file is a common operation and doing this
without loading the whole file into memory is a typical
requirement.

The standard library has extensions to the Web
Crypto API that are useful when doing things
like hashing a file. These can be accessed through the
`crypto` module, a drop-in replacement for the Web Crypto
API that delegates to the native implementation when
possible.


In [5]:
import { crypto } from "jsr:@std/crypto";
const file = await Deno.open("09_Cryptography.ipynb", { read: true });

We obtain an async iterable using the readable property.


In [6]:
const readableStream = file.readable;


This time, when we call `crypto.subtle.digest`, we're using the
imported version that allows us to operate on the
async iterable.


In [7]:
const fileHashBuffer = await crypto.subtle.digest("SHA-256", readableStream);


Finally, we obtain the hex result using toHashString like earlier.


In [8]:
const fileHash = toHashString(fileHashBuffer);
console.log(fileHash);


d5f47b35c2b63fe0c066a3acbc37f460c64e0338fb7081c4ef9e7e09a8300aba


## Generating & Validating UUIDs

UUIDs (universally unique identifier) can be used to uniquely identify some object or data.


A random UUID can be generated using the builtin Web Cryptography API. This type of UUID is also known as UUID v4.


In [9]:
const myUUID = crypto.randomUUID();
myUUID;


"42629b95-a84c-4339-b404-80a05c17e5b1"

The standard library contains some more functions for working with UUIDs.


In [10]:
import * as uuid from "jsr:@std/uuid";


You can validate that a given string is a valid UUID.


In [11]:
uuid.validate("not a UUID");


false

In [12]:
uuid.validate("6ec0bd7f-11c0-43da-975e-2a8ad9ebae0b");


true

You can also generate a time-based (v1) UUID. By default this uses system time as the time source.


In [13]:
uuid.v1.generate();


"5fe4e420-1475-11ef-ae98-c303dc00e183"

SHA-1 namespaced (v5) UUIDs can also be generated. For this you need to specify a namespace and data:


In [14]:
const NAMESPACE_URL = "6ba7b810-9dad-11d1-80b4-00c04fd430c8";

const data = new TextEncoder().encode("deno.land");
await uuid.v5.generate(NAMESPACE_URL, data);


"21bbd59d-9801-53ce-8902-9559de3ebd39"

## ULID

One common need for distributed systems are identifiers. ULIDs are a universally unique lexicographically sortable identifier with some nice properties. They are 128-bit values, encoded as 26 character strings which also encode the timestamp. They play very nicely with Deno KV.


The standard library contains a function for generating ULIDs.


In [15]:
import { ulid } from "jsr:@std/ulid";


To generate a ULID, simply call the function.


In [16]:
ulid();


"01HY3TJYVX62VNYRCYGTBYB55M"

ULIDs can also be generated from a timestamp. This is useful for migrating from another system.


In [17]:
const timestamp = Date.now();
ulid(timestamp);


"01HY3TJYW6XXPDX38SCQFMT9RA"

Given a ULID, you can get the timestamp back out


In [18]:
import { decodeTime } from "jsr:@std/ulid";
const myULID = ulid();
decodeTime(myULID);

1715967982479

Optionally, if you're not on a distributed system and want monotonic ULIDs, you can use the monotonic ULID generator instead.


In [19]:
import { monotonicUlid } from "jsr:@std/ulid";

for (let i = 0; i < 3; i++) {
  console.log(monotonicUlid());
}


01HY3TJYWQ2VZJXSVE3EC4VD0F
01HY3TJYWQ2VZJXSVE3EC4VD0G
01HY3TJYWQ2VZJXSVE3EC4VD0H
